<a href="https://colab.research.google.com/github/mamuncseru/exploring_transformer/blob/main/Masked_Language_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.1 MB/s eta 0:00:00


In [3]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2023-05-12 05:23:23--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  4.69MB/s    in 1.0s    

2023-05-12 05:23:25 (4.69 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import textwrap
from pprint import pprint

from transformers import pipeline

In [7]:
df = pd.read_csv('bbc_text_cls.csv')

In [8]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [10]:
labels = set(df['labels'])
labels

{'business', 'entertainment', 'politics', 'sport', 'tech'}

In [15]:
# pick a label
label = 'business'

In [16]:
texts = df[df['labels'] == label]['text']
texts.head()

0    Ad sales boost Time Warner profit\n\nQuarterly...
1    Dollar gains on Greenspan speech\n\nThe dollar...
2    Yukos unit buyer faces loan claim\n\nThe owner...
3    High fuel prices hit BA's profits\n\nBritish A...
4    Pernod takeover talk lifts Domecq\n\nShares in...
Name: text, dtype: object

In [17]:
np.random.seed(1234)

In [18]:
i = np.random.choice(texts.shape[0])
doc = texts.iloc[i]

In [19]:
print(textwrap.fill(doc, replace_whitespace=False, fix_sentence_endings=True))

Bombardier chief to leave company

Shares in train and plane-making
giant Bombardier have fallen to a 10-year low following the departure
of its chief executive and two members of the board.

Paul Tellier,
who was also Bombardier's president, left the company amid an ongoing
restructuring.  Laurent Beaudoin, part of the family that controls the
Montreal-based firm, will take on the role of CEO under a newly
created management structure.  Analysts said the resignations seem to
have stemmed from a boardroom dispute.  Under Mr Tellier's tenure at
the company, which began in January 2003, plans to cut the worldwide
workforce of 75,000 by almost a third by 2006 were announced.  The
firm's snowmobile division and defence services unit were also sold
and Bombardier started the development of a new aircraft seating 110
to 135 passengers.

Mr Tellier had indicated he wanted to stay at the
world's top train maker and third largest manufacturer of civil
aircraft until the restructuring was comple

In [20]:
mlm = pipeline('fill-mask')

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [21]:
mlm('Bombardier cheif to leave <mask>')

[{'score': 0.06977901607751846,
  'token': 1470,
  'token_str': ' France',
  'sequence': 'Bombardier cheif to leave France'},
 {'score': 0.041473690420389175,
  'token': 2201,
  'token_str': ' Paris',
  'sequence': 'Bombardier cheif to leave Paris'},
 {'score': 0.03459375351667404,
  'token': 558,
  'token_str': ' office',
  'sequence': 'Bombardier cheif to leave office'},
 {'score': 0.03167286142706871,
  'token': 9313,
  'token_str': ' »',
  'sequence': 'Bombardier cheif to leave »'},
 {'score': 0.027178337797522545,
  'token': 2627,
  'token_str': ' Italy',
  'sequence': 'Bombardier cheif to leave Italy'}]

In [22]:
text = 'Shares in <mask> and plane-making' + \
  'giant Bombardier have fallen to a 10-year low following the departure ' + \
  'of its cheif executive and two members of the board.'

mlm(text)

[{'score': 0.6846084594726562,
  'token': 11016,
  'token_str': ' Airbus',
  'sequence': 'Shares in Airbus and plane-makinggiant Bombardier have fallen to a 10-year low following the departure of its cheif executive and two members of the board.'},
 {'score': 0.2412230372428894,
  'token': 6722,
  'token_str': ' Boeing',
  'sequence': 'Shares in Boeing and plane-makinggiant Bombardier have fallen to a 10-year low following the departure of its cheif executive and two members of the board.'},
 {'score': 0.010099587962031364,
  'token': 14833,
  'token_str': ' Renault',
  'sequence': 'Shares in Renault and plane-makinggiant Bombardier have fallen to a 10-year low following the departure of its cheif executive and two members of the board.'},
 {'score': 0.007334011606872082,
  'token': 1470,
  'token_str': ' France',
  'sequence': 'Shares in France and plane-makinggiant Bombardier have fallen to a 10-year low following the departure of its cheif executive and two members of the board.'},


In [24]:
text = 'Shares in train and plane-making' + \
  'giant Bombardier have fallen to a 10-year low following the <mask> ' + \
  'of its cheif executive and two members of the board.'

pprint(mlm(text))

[{'score': 0.536346435546875,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the resignation of its cheif executive '
              'and two members of the board.',
  'token': 6985,
  'token_str': ' resignation'},
 {'score': 0.18109850585460663,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departure of its cheif executive '
              'and two members of the board.',
  'token': 5824,
  'token_str': ' departure'},
 {'score': 0.11224262416362762,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departures of its cheif executive '
              'and two members of the board.',
  'token': 25624,
  'token_str': ' departures'},
 {'score': 0.05888812616467476,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year lo

In [25]:
text = 'Shares in train and plane-making' + \
  'giant Bombardier have fallen to a 10-year low following the departure ' + \
  'of its cheif <mask> and two members of the board.'

pprint(mlm(text))

[{'score': 0.09341490268707275,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departure of its cheifier and two '
              'members of the board.',
  'token': 906,
  'token_str': 'ier'},
 {'score': 0.05520506203174591,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departure of its cheifaires and two '
              'members of the board.',
  'token': 25082,
  'token_str': 'aires'},
 {'score': 0.04195832461118698,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departure of its cheifers and two '
              'members of the board.',
  'token': 268,
  'token_str': 'ers'},
 {'score': 0.04069560766220093,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departure of its cheifolder 

In [26]:
text = 'Shares in train and plane-making' + \
  'giant Bombardier have fallen to a 10-year low following the departure ' + \
  'of its cheif executive and two <mask> of the board.'

pprint(mlm(text))

[{'score': 0.8848996758460999,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departure of its cheif executive '
              'and two members of the board.',
  'token': 453,
  'token_str': ' members'},
 {'score': 0.07351367920637131,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departure of its cheif executive '
              'and two thirds of the board.',
  'token': 29193,
  'token_str': ' thirds'},
 {'score': 0.01811094395816326,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departure of its cheif executive '
              'and two directors of the board.',
  'token': 5392,
  'token_str': ' directors'},
 {'score': 0.003917124588042498,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low followi